In [168]:
# imports
import pandas as pd
import numpy as np
import string
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
#--------#
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iforrest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iforrest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\iforrest\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [169]:
# define stopwords
stopwords = stopwords.words('english')

In [170]:
# load dataframes
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [171]:
# check train_df
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [172]:
# check test_df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [173]:
# create text preprocessing function
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords\
              and token != " " \
              and token.strip() not in string.punctuation.replace('#', '')]
    text = " ".join(tokens)
    return text

In [174]:
# create vectorizer
count_vectorizer = feature_extraction.text.CountVectorizer()
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(min_df=0.01, max_df=.95, ngram_range=(1,3))

In [175]:
# create list of train_text and test_text
train_text = list(train_df['text'])
test_text = list(test_df['text'])

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text.split(' ')]
    lemm_text = ' '.join(lemm_text)
    lemm_text = re.sub(r"\# ", "#", lemm_text)
            
    return lemm_text

In [176]:
# create list of texts with punctuation removed (replaced with single space)
new_train_text = []
new_test_text = []

# create translator to replace punctuation with spaces
# better score generated with punctuation removed
# string of punctuation with hashtag '#' removed - popular and specific use on Twitter
punctuation_no_hash = string.punctuation.replace('#','')
punctuation_no_hash
translator = str.maketrans(punctuation_no_hash, ' '*len(punctuation_no_hash)) #map punctuation to space
# translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space

# apply to training text
for text in train_text:
#     new_text = text.translate(translator)
    new_text = preprocess_text(text)
    # lemmatize text
    new_train_text.append(new_text)
    
# apply to test text
for text in test_text:
    print(text)
    # new_text = text.translate(translator)
    new_text = preprocess_text(text)
    print(new_text)
    print(new_text)
    new_test_text.append(new_text)
    print('\n')

Just happened a terrible car crash
happened terrible car crash
happened terrible car crash


Heard about #earthquake is different cities, stay safe everyone.
heard # earthquake different cities stay safe everyone
heard # earthquake different cities stay safe everyone


there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
forest fire spot pond geese fleeing across street save
forest fire spot pond geese fleeing across street save


Apocalypse lighting. #Spokane #wildfires
apocalypse lighting # spokane # wildfires
apocalypse lighting # spokane # wildfires


Typhoon Soudelor kills 28 in China and Taiwan
typhoon soudelor kills 28 china taiwan
typhoon soudelor kills 28 china taiwan


We're shaking...It's an earthquake
're shaking ... 's earthquake
're shaking ... 's earthquake


They'd probably still show more life than Arsenal did yesterday, eh? EH?
'd probably still show life arsenal yesterday eh eh
'd probably still show life arsenal yesterday 

@GACourts If 90BLKs&amp;8WHTs colluded 2 take WHT F @USAgov AUTH Hostage&amp;2 make her look BLK w/Bioterrorism&amp;use her lgl/org IDis ID still hers?
gacourts 90blks amp 8whts colluded 2 take wht f usagov auth hostage amp 2 make look blk w/bioterrorism amp use lgl/org idis id still
gacourts 90blks amp 8whts colluded 2 take wht f usagov auth hostage amp 2 make look blk w/bioterrorism amp use lgl/org idis id still


Bioterrorism: Guidelines for Medical and Public Health Management Donald A. Hen http://t.co/tDg0Bsmd6c http://t.co/iirWFZMKHO
bioterrorism guidelines medical public health management donald a. hen http //t.co/tdg0bsmd6c http //t.co/iirwfzmkho
bioterrorism guidelines medical public health management donald a. hen http //t.co/tdg0bsmd6c http //t.co/iirwfzmkho


To fight bioterrorism sir.
fight bioterrorism sir
fight bioterrorism sir


To fight bioterrorism sir.
fight bioterrorism sir
fight bioterrorism sir


@prsnvns that's good to hear. Mine could be better but I can manage.


@chistate33 @thehill What hate crimes? It's the looney libs that are always burning down buildings and vandalzing private property
chistate33 thehill hate crimes 's looney libs always burning buildings vandalzing private property
chistate33 thehill hate crimes 's looney libs always burning buildings vandalzing private property


3 rules for a long life.  #1ÛÒDonÛªt go in burning buildings! #2ÛÒDonÛªt jump out of perfectly good airplanes! #3ÛÒDonÛªt awaken sleeping giants!
3 rules long life # 1ûòdonûªt go burning buildings # 2ûòdonûªt jump perfectly good airplanes # 3ûòdonûªt awaken sleeping giants
3 rules long life # 1ûòdonûªt go burning buildings # 2ûòdonûªt jump perfectly good airplanes # 3ûòdonûªt awaken sleeping giants


Port Coquitlam fire burning several businesses http://t.co/Q5FzYLBvTk
port coquitlam fire burning several businesses http //t.co/q5fzylbvtk
port coquitlam fire burning several businesses http //t.co/q5fzylbvtk


@speechgirlAM On par with caring f



This is hilarious! Look at all of you! We got: Hairity Rainbow Crash Spitty Pie #Spikebot
hilarious look got hairity rainbow crash spitty pie # spikebot
hilarious look got hairity rainbow crash spitty pie # spikebot


Thieves crash dealership gate steal four vehicles in Killeen http://t.co/aA47AXC6sr http://t.co/fz5axahPVr
thieves crash dealership gate steal four vehicles killeen http //t.co/aa47axc6sr http //t.co/fz5axahpvr
thieves crash dealership gate steal four vehicles killeen http //t.co/aa47axc6sr http //t.co/fz5axahpvr


Time to enjoy summer in a crash course of two weeks... I will never take summer classes again. (That's a lie)
time enjoy summer crash course two weeks ... never take summer classes 's lie
time enjoy summer crash course two weeks ... never take summer classes 's lie


#portmoody Clarke Rd hill n/b blocked by crash past Seaview traffic back to Kemsley. Take Como Lake to Gatensbury to get around it.
# portmoody clarke rd hill n/b blocked crash past seaview traff


Six Palestinians Kidnapped in West Bank Hebron Home Demolished - International Middle East Media Center http://t.co/J9UzIIQ3PX
six palestinians kidnapped west bank hebron home demolished international middle east media center http //t.co/j9uziiq3px
six palestinians kidnapped west bank hebron home demolished international middle east media center http //t.co/j9uziiq3px


Uttarakhand: Journalist's house demolished who exposed corruption - Oneindia http://t.co/Ro1aWWuDYE
uttarakhand journalist 's house demolished exposed corruption oneindia http //t.co/ro1awwudye
uttarakhand journalist 's house demolished exposed corruption oneindia http //t.co/ro1awwudye


@ILoveNuiHarime More than that.
Demolished.
ilovenuiharime demolished
ilovenuiharime demolished


Before &amp; After #underconstruction #demolished #Melbourne #Residential #commercial #builder #luxuryhomes #townhouses ?? http://t.co/6iD7CZvMEd
amp # underconstruction # demolished # melbourne # residential # commercial # builder # luxu

...

families sue legionnaires 40 families affected fatal outbreak legionnaires disea ... http //t.co/bozfh1m3wb
families sue legionnaires 40 families affected fatal outbreak legionnaires disea ... http //t.co/bozfh1m3wb


Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/6oPYSf87K2
families sue legionnaires 40 families affected fatal outbreak legionnaires disea ... http //t.co/6opysf87k2
families sue legionnaires 40 families affected fatal outbreak legionnaires disea ... http //t.co/6opysf87k2


More than 40 families affected by the fatal outbreak of Legionnaires' disease in Edinburgh areÛ_ http://t.co/XpvKqME5Ls
40 families affected fatal outbreak legionnaires disease edinburgh areû_ http //t.co/xpvkqme5ls
40 families affected fatal outbreak legionnaires disease edinburgh areû_ http //t.co/xpvkqme5ls


Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) http://t.co/s9fJTsGbmd
pandemonium aba woman d



I'm having a meltdown because of Game of Thrones ?? literally cried screamed and threw my computer #WHYYYY #redwedding
'm meltdown game thrones literally cried screamed threw computer # whyyyy # redwedding
'm meltdown game thrones literally cried screamed threw computer # whyyyy # redwedding


I just screamed when I saw Jordan come out the door behind Donnie on #Wahlburgers  ??
screamed saw jordan come door behind donnie # wahlburgers
screamed saw jordan come door behind donnie # wahlburgers


@joshshatsays I JUST SCREAMED TO MYSELF BECAUSE YEAH
joshshatsays screamed yeah
joshshatsays screamed yeah


OH MY GOSH IM AT MY AUNTS HOUSE AND THIS POST IT WAS ON HER COUNTER AND I SCREAMED BC I THOUGHT IT SAID CHRIS KELLER http://t.co/DNofWtAkAt
oh gosh im aunts house post counter screamed bc thought said chris keller http //t.co/dnofwtakat
oh gosh im aunts house post counter screamed bc thought said chris keller http //t.co/dnofwtakat


@theboysftvines oh my god i SCREAMED WHEN I SAW THIS Y

must-see massive isis suicide truck bombing caught tape http //t.co/hmspypozln via gabbyogbechie1tpgazette


According to the tabloids Cilla could've been saved. In the sense that Titanic wouldn't have sunk had it avoided the iceberg.
according tabloids cilla could 've saved sense titanic would n't sunk avoided iceberg
according tabloids cilla could 've saved sense titanic would n't sunk avoided iceberg


Managed to shuffle over to the bed. Butterflies swarmed in the pit of her belly at @VengefulKnave's offer. Her canines sunk into her lower++
managed shuffle bed butterflies swarmed pit belly vengefulknave 's offer canines sunk lower++
managed shuffle bed butterflies swarmed pit belly vengefulknave 's offer canines sunk lower++


Today 5/8/1941 Today 1941 British cargo ship Kumasian torpedoed/sunk Atlantic lost 1 of 60 lives Any1 have link? #crewlist #WW2
today 5/8/1941 today 1941 british cargo ship kumasian torpedoed/sunk atlantic lost 1 60 lives any1 link # crewlist # ww2
today 5/8/1

ibautumn 'm traumatised ca n't even spell properly excuse typos
ibautumn 'm traumatised ca n't even spell properly excuse typos


@TitusOReily He's not responsible for the trouble urine!
titusoreily 's responsible trouble urine
titusoreily 's responsible trouble urine


Pissed something off in the wood pile. Pulling pieces out and I get a small hiss and a series of low clicks for my trouble.
pissed something wood pile pulling pieces get small hiss series low clicks trouble
pissed something wood pile pulling pieces get small hiss series low clicks trouble


RT MMDA: ADVISORY: Stalled Bus at EDSA Service Road Cubao SB due to mechanical trouble as of 7:53 AM. 1 lane occupied. MMDA T/C on site. TÛ_
rt mmda advisory stalled bus edsa service road cubao sb due mechanical trouble 7:53 1 lane occupied mmda t/c site tû_
rt mmda advisory stalled bus edsa service road cubao sb due mechanical trouble 7:53 1 lane occupied mmda t/c site tû_


a room full of adults going ÛÏboonew boonewÛ along w

'police officer wounded suspect dead exchanging shots http //t.co/gunq7ztun4 #
'police officer wounded suspect dead exchanging shots http //t.co/gunq7ztun4 #


Police Officer Wounded Suspect Dead After Exchanging Shots: Richmond police officer wounded suspect killed a... http://t.co/YIKQRqczVZ
police officer wounded suspect dead exchanging shots richmond police officer wounded suspect killed ... http //t.co/yikqrqczvz
police officer wounded suspect dead exchanging shots richmond police officer wounded suspect killed ... http //t.co/yikqrqczvz


He was crushed for our iniquities; the punishment that brought us peace was upon him &amp; by his wounds we are healed. -Isa 53:5
crushed iniquities punishment brought us peace upon amp wounds healed -isa 53:5
crushed iniquities punishment brought us peace upon amp wounds healed -isa 53:5


@bkanioros Lizeth and I tried over and over again to fix our relationship it never worked bk you cannot heal those wounds I love you for
bkanioros lizeth tri

In [177]:
# vectorize text in train data
train_vectors = count_vectorizer.fit_transform(train_df["text"])
# vectorize text in test data
test_vectors = count_vectorizer.transform(test_df["text"])

In [178]:
# alternate train vectors - tfidf, punctuation removed
train_vectors_tfidf = tfidf_vectorizer.fit_transform(new_train_text)
train_vectors_tfidf

<7613x80 sparse matrix of type '<class 'numpy.float64'>'
	with 21279 stored elements in Compressed Sparse Row format>

In [179]:
# create basic model assuming tweet text is proper indicator of disaster
clf = linear_model.RidgeClassifier()
# create second logistic regression model
lr = linear_model.LogisticRegression()
# create decisiontreeregressor model
dtc = DecisionTreeClassifier()
# create SVC Classifier
svc = SVC()

In [183]:
# check initial cross-validation score for RidgeClassifier
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
print(scores)

[0.59421842 0.56498283 0.64051005]


In [184]:
# check cross validation scores on tfidf vectorizer
# tokenizing text, removing stop words, and removing punctuation increases scores
scores = model_selection.cross_val_score(clf, train_vectors_tfidf, train_df["target"], cv=3, scoring="f1")
print(scores)

[0.61826698 0.61220743 0.65178571]


In [185]:
# check scores using SVC Classifier
scores = model_selection.cross_val_score(svc, train_vectors_tfidf, train_df["target"], cv=3, scoring="f1")
print(scores)

[0.59593456 0.58683206 0.66071429]


In [186]:
# shuffle train text and target together
temp = list(zip(new_train_text, list(train_df['target'])))
random.shuffle(temp)
new_train_text_shuffled, train_target_shuffled = zip(*temp)

# revectorize shuffled data
train_vectors_tfidf_shuffled = tfidf_vectorizer.fit_transform(new_train_text)

# obtain scores of shuffled data
scores = model_selection.cross_val_score(clf, train_vectors_tfidf_shuffled, train_target_shuffled, cv=3, scoring="f1")
print(scores)

[0.18142235 0.15176559 0.11670663]


In [52]:
# check cross validation score on tfidf vectorizer and decision tree classifier
scores = model_selection.cross_val_score(dtc, train_vectors_tfidf, train_df["target"], cv=3, scoring="f1")
print(scores)

[0.55717054 0.55628518 0.62739965]


In [51]:
# fit ridge classifier to train vectors and train target variable
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [17]:
# create sample submission dataframe
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission["target"] = clf.predict(test_vectors)
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [38]:
# export submission
sample_submission.to_csv("./data/submission.csv", index=False)